In [ ]:
using LinearAlgebra
using BenchmarkTools

In [ ]:
#=
subroutine dsyev	(
character 	JOBZ,
character 	UPLO,
integer 	N,
double precision, dimension( lda, * ) 	A,
integer 	LDA,
double precision, dimension( * ) 	W,
double precision, dimension( * ) 	WORK,
integer 	LWORK,
integer 	INFO 
)
=#

function dsyev!(jobz, uplo, n, A, lda, w, work, lwork, info)
    ccall((:dsyev_64_, "libopenblas64_"), Cvoid,
        (Ref{UInt8}, Ref{UInt8}, Ref{Int}, Ptr{Float64}, Ref{Int}, 
            Ptr{Float64}, Ptr{Float64}, Ref{Int}, Ptr{Int}),
        jobz, uplo, n, A, lda, w, work, lwork, info)
end

function mydsyev!(A)
    n = size(A, 1)
    jobz = 'V'
    lda = n
    w = zeros(n)
    info = Ref{Int}(0)
    
    # Perform an optimal workspace query
    lwork = -1
    work = zeros(1)
    dsyev!(jobz, A.uplo, n, A.data, lda, w, work, lwork, info)
    lwork = Int(real(work[1]))
    resize!(work, lwork)

    @time dsyev!(jobz, A.uplo, n, A.data, lda, w, work, lwork, info)
    
    return w, A.data
end

mydsyev(A) = mydsyev!(copy(A))

In [ ]:
#=
subroutine dsyevd	(
character 	JOBZ,
character 	UPLO,
integer 	N,
double precision, dimension( lda, * ) 	A,
integer 	LDA,
double precision, dimension( * ) 	W,
double precision, dimension( * ) 	WORK,
integer 	LWORK,
integer, dimension( * ) 	IWORK,
integer 	LIWORK,
integer 	INFO 
)
=#

function dsyevd!(jobz, uplo, n, A, lda, 
        w, work, lwork, iwork, liwork, info)
    ccall((:dsyevd_64_, "libopenblas64_"), Cvoid,
        (Ref{UInt8}, Ref{UInt8}, Ref{Int}, 
            Ptr{Float64}, Ref{Int}, Ptr{Float64},
            Ptr{Float64}, Ref{Int}, Ptr{Int}, Ref{Int}, Ptr{Int}),
        jobz, uplo, n, 
        A, lda, w,
        work, lwork, iwork, liwork, info)
end

function mydsyevd!(A)
    n = size(A, 1)
    jobz = 'V'
    lda = n
    w = zeros(n)
    info = Ref{Int}(0)
    
    # Perform an optimal workspace query
    lwork = -1
    liwork = -1
    work = zeros(1)
    iwork  = zeros(Int, 1)
    dsyevd!(jobz, A.uplo, n, A.data, lda, w,
        work, lwork, iwork, liwork, info)
    lwork = Int(real(work[1]))
    liwork = iwork[1]
    resize!(work, lwork)
    resize!(iwork, liwork)

    @time dsyevd!(jobz, A.uplo, n, A.data, lda, w,
        work, lwork, iwork, liwork, info)
    
    return w, A.data
end

mydsyevd(A) = mydsyevd!(copy(A))

In [ ]:
#=
subroutine dsyevr	(	character 	JOBZ,
character 	RANGE,
character 	UPLO,
integer 	N,
double precision, dimension( lda, * ) 	A,
integer 	LDA,
double precision 	VL,
double precision 	VU,
integer 	IL,
integer 	IU,
double precision 	ABSTOL,
integer 	M,
double precision, dimension( * ) 	W,
double precision, dimension( ldz, * ) 	Z,
integer 	LDZ,
integer, dimension( * ) 	ISUPPZ,
double precision, dimension( * ) 	WORK,
integer 	LWORK,
integer, dimension( * ) 	IWORK,
integer 	LIWORK,
integer 	INFO 
)
=#

function dsyevr!(jobz, range, uplo, n, A, lda,
        vl, vu, il, iu, abstol, m, w, Z, ldz, isuppz,
        work, lwork, iwork, liwork, info)
    ccall((:dsyevr_64_, "libopenblas64_"), Cvoid,
        (Ref{UInt8}, Ref{UInt8}, Ref{UInt8}, Ref{Int},
            Ptr{Float64}, Ref{Int}, 
            Ref{Float64}, Ref{Float64}, Ref{Int}, Ref{Int}, 
            Ref{Float64}, Ptr{Int},
            Ptr{Float64}, Ptr{Float64}, Ref{Int}, Ptr{Int},
            Ptr{Float64}, Ref{Int}, Ptr{Int}, Ref{Int},
            Ptr{Int}),
        jobz, range, uplo, n, 
        A, lda,
        vl, vu, il, iu, 
        abstol, m, 
        w, Z, ldz, isuppz,
        work, lwork, iwork, liwork, info)
end

function mydsyevr!(A)
    n = size(A, 1)
    jobz = 'V'
    range = 'V'
    lda = n
    vl = 0.0
    vu = Inf
    il = 0
    iu = 0
    abstol = -1.0
    m = Ref{Int}(0)
    w = zeros(n)
    Z = zeros(n, n)
    ldz = n
    isuppz = zeros(Int, 2n)
    info = Ref{Int}(0)
    
    # Perform an optimal workspace query
    lwork = -1
    liwork = -1
    work = zeros(1)
    iwork  = zeros(Int, 1)
    dsyevr!(jobz, range, A.uplo, n, A.data, lda,
        vl, vu, il, iu, abstol, m, w, Z, ldz, isuppz,
        work, lwork, iwork, liwork, info)
    lwork = Int(real(work[1]))
    liwork = iwork[1]
    resize!(work, lwork)
    resize!(iwork, liwork)

    @time dsyevr!(jobz, range, A.uplo, n, A.data, lda,
        vl, vu, il, iu, abstol, m, w, Z, ldz, isuppz,
        work, lwork, iwork, liwork, info)
    
    return w[1:m[]], Z[:,1:m[]]
end

mydsyevr(A) = mydsyevr!(copy(A))

In [ ]:
#=
subroutine dsyevx	(
character 	JOBZ,
character 	RANGE,
character 	UPLO,
integer 	N,
double precision, dimension( lda, * ) 	A,
integer 	LDA,
double precision 	VL,
double precision 	VU,
integer 	IL,
integer 	IU,
double precision 	ABSTOL,
integer 	M,
double precision, dimension( * ) 	W,
double precision, dimension( ldz, * ) 	Z,
integer 	LDZ,
double precision, dimension( * ) 	WORK,
integer 	LWORK,
integer, dimension( * ) 	IWORK,
integer, dimension( * ) 	IFAIL,
integer 	INFO 
)
=#

function dsyevx!(jobz, range, uplo, n, A, lda,
        vl, vu, il, iu, abstol, m, w, Z, ldz,
        work, lwork, iwork, ifail, info)
    ccall((:dsyevx_64_, "libopenblas64_"), Cvoid,
        (Ref{UInt8}, Ref{UInt8}, Ref{UInt8}, Ref{Int},
            Ptr{Float64}, Ref{Int}, 
            Ref{Float64}, Ref{Float64}, Ref{Int}, Ref{Int}, 
            Ref{Float64}, Ptr{Int},
            Ptr{Float64}, Ptr{Float64}, Ref{Int},
            Ptr{Float64}, Ref{Int}, Ptr{Int}, Ptr{Int},
            Ptr{Int}),
        jobz, range, uplo, n, A, lda,
        vl, vu, il, iu, abstol, m, w, Z, ldz,
        work, lwork, iwork, ifail, info)
end

function mydsyevx!(A)
    n = size(A, 1)
    jobz = 'V'
    range = 'V'
    lda = n
    vl = 0.0
    vu = Inf
    il = 0
    iu = 0
    abstol = -1.0
    m = Ref{Int}(0)
    w = zeros(n)
    Z = zeros(n, n)
    ldz = n
    work   = zeros(1)
    iwork  = zeros(Int, 5n)
    ifail  = zeros(Int, n)
    info = Ref{Int}(0)
    
    # Perform an optimal workspace query
    lwork = -1
    liwork = -1
    work = zeros(1)
    dsyevx!(jobz, range, A.uplo, n, A.data, lda,
        vl, vu, il, iu, abstol, m, w, Z, ldz,
        work, lwork, iwork, ifail, info)
    lwork = Int(real(work[1]))
    resize!(work, lwork)

    @time dsyevx!(jobz, range, A.uplo, n, A.data, lda,
        vl, vu, il, iu, abstol, m, w, Z, ldz,
        work, lwork, iwork, ifail, info)
    
    return w[1:m[]], Z[:,1:m[]]
end

mydsyevx(A) = mydsyevx!(copy(A))

---

In [ ]:
n = 2000
A = Symmetric(rand(n,n));

In [ ]:
@time F = eigen(A);

In [ ]:
#w, V = mydsyev(A);
w, V = mydsyevd(A);
w, V = mydsyevr(A);
w, V = mydsyevx(A);

In [ ]:
norm(F.values[F.values.>0] - w)/n

In [ ]:
norm(A*V - V*Diagonal(w))/n

---

In [ ]:
function blastest(func; n=1000)
    A = Symmetric(rand(n,n))
    t = zeros(4)
    for i = 1:4
        BLAS.set_num_threads(i)
        t[i] = @elapsed w, V = func(A);
    end
    BLAS.set_num_threads(2)
    t./minimum(t)
end

In [ ]:
blastest(mydsyevd)

---